In [1]:
import pandas as pd
import numpy as np
import requests
import json
from src.config.secrets import settings

In [2]:
# base Url and API key
url = settings.API_BASE_URL
api_key = settings.API_KEY

# Headers
headers = {
    'x-apisports-key': api_key,
    }


# Qeury
league_param = {
    "id": "39",
    "name": "Premier League",
    "code": "GB-ENG",
    "season": 2024,
    "current": False,
    "type": "league",
    "country": "England"}


team_param = {
    "league": "39",
    "season": "2024"
}


team_stats_param = {
    "league": "39",
    "season": 2024,
    'team': "33"
}


players_param = {
    "league": "39",
    "season": "2024",
}

In [3]:
print("Request Url :" + url)

Request Url : https://v3.football.api-sports.io/


In [4]:
# League Request
response_l = requests.get(url= url+"leagues", headers=headers, params=league_param)
print(response_l.status_code)

200


In [5]:
league_data = response_l.json()
league_data.keys()

dict_keys(['get', 'parameters', 'errors', 'results', 'paging', 'response'])

In [6]:
league_data['response']

[{'league': {'id': 39,
   'name': 'Premier League',
   'type': 'League',
   'logo': 'https://media.api-sports.io/football/leagues/39.png'},
  'country': {'name': 'England',
   'code': 'GB-ENG',
   'flag': 'https://media.api-sports.io/flags/gb-eng.svg'},
  'seasons': [{'year': 2024,
    'start': '2024-08-16',
    'end': '2025-05-25',
    'current': False,
    'coverage': {'fixtures': {'events': True,
      'lineups': True,
      'statistics_fixtures': True,
      'statistics_players': True},
     'standings': True,
     'players': True,
     'top_scorers': True,
     'top_assists': True,
     'top_cards': True,
     'injuries': True,
     'predictions': True,
     'odds': False}}]}]

In [19]:
# Teams Request
response_t = requests.get(url= url+"teams", headers=headers, params=team_param)
print(response_t.status_code)

MissingSchema: Invalid URL 'v3.football.api-sports.ioteams': No scheme supplied. Perhaps you meant https://v3.football.api-sports.ioteams?

In [30]:
team_dict = response_t.json()
team_dict.keys()

dict_keys(['get', 'parameters', 'errors', 'results', 'paging', 'response'])

In [36]:
team_data = team_dict['response']
team_df = pd.DataFrame([team_data[i]['team'] for i in range(len(team_data))])
team_df

,id,name,code,country,founded,national,logo
0,33,Manchester United,MUN,England,1878,False,https://media.api-sports.io/football/teams/33.png
1,34,Newcastle,NEW,England,1892,False,https://media.api-sports.io/football/teams/34.png
2,35,Bournemouth,BOU,England,1899,False,https://media.api-sports.io/football/teams/35.png
3,36,Fulham,FUL,England,1879,False,https://media.api-sports.io/football/teams/36.png
4,39,Wolves,WOL,England,1877,False,https://media.api-sports.io/football/teams/39.png
5,40,Liverpool,LIV,England,1892,False,https://media.api-sports.io/football/teams/40.png
6,41,Southampton,SOU,England,1885,False,https://media.api-sports.io/football/teams/41.png
7,42,Arsenal,ARS,England,1886,False,https://media.api-sports.io/football/teams/42.png
8,45,Everton,EVE,England,1878,False,https://media.api-sports.io/football/teams/45.png
9,46,Leicester,LEI,England,1884,False,https://media.api-sports.io/football/teams/46.png


In [4]:
# Player data Request
response_ts = requests.get(url=  url+"teams/statistics", headers=headers, params=team_stats_param)
print(response_ts.status_code)

200


In [5]:
stats_data = response_ts.json()
stats_data.keys()

dict_keys(['get', 'parameters', 'errors', 'results', 'paging', 'response'])

In [7]:
stats = stats_data['response']
print(type(stats))
#stats.keys()

<class 'dict'>


In [13]:
# For team id
stats['team']['id']

# for form
stats['form']

#for fixtures
stats['fixtures']['wins']['home']
stats['fixtures']['wins']['away']
stats['fixtures']['loses']['home']
stats['fixtures']['loses']['away']
stats['fixtures']['draws']['home']
stats['fixtures']['draws']['away']

# for goals
stats['goals']['for']['total']['home']
stats['goals']['for']['total']['away']
stats['goals']['against']['total']['home']
stats['goals']['against']['total']['away']

# for clean sheets
stats['clean_sheet']['home']
stats['clean_sheet']['away']

# for failed to score
stats['failed_to_score']['home']
stats['failed_to_score']['away']

# for penalties
stats['penalty']['scored']['total']
stats['penalty']['missed']['total']

# for cards
sum((bucket.get("total") or 0) for bucket in stats['cards']['yellow'].values())
sum((bucket.get("total") or 0) for bucket in stats['cards']['red'].values())

3

In [12]:
# stats is your dict, e.g. stats["cards"]["yellow"]["0-15"]["total"]

yellow_totals_sum = sum((bucket.get("total") or 0) for bucket in stats['cards']['yellow'].values())

print(yellow_totals_sum)

86


In [60]:
def get_teams(season, league=39):
    team_param = {
    "league": league,
    "season": season
    }
    response_t = requests.get(url= url+"teams", headers=headers, params=team_param)
    if response_t.status_code == 200:
        team_dict = response_t.json()
        team_data = team_dict['response']
        team_df = pd.DataFrame([team_data[i]['team'] for i in range(len(team_data))])
        return team_df
    else:
        print(f"Error: {response_t.status_code}")
        return None

In [ ]:
def get_team_stats(team_id, season=2024, league_id=39):
    team_stats_param = {
        "league": "39",
        "season": season,
        'team': team_id
    }
    response_ts = requests.get(url= url+"teams/statistics", headers=headers, params=team_stats_param)
    if response_ts.status_code == 200:
        stats_data = response_ts.json()
        return stats_data['response']
    else:
        print(f"Failed to retrieve data for team ID {team_id}. Status code: {response_ts.status_code}")
        return None